# Análisis de datos de distribución de combustible 2017 - 2019

In [161]:
import os
import os.path as op
import pandas as pd
import numpy as np

In [162]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap

output_notebook()

Loading BokehJS ...

In [163]:
HOUR = np.timedelta64(1, 'h')
MIN = np.timedelta64(1, 'm')
SEC = np.timedelta64(1, 's')

In [164]:
def read_file(fname, sheet_name):
    week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    
    df = pd.read_excel(fname, sheet_name=sheet_name)
    df['dia'] = df['FECHA'].dt.day_name()
    df['dia'] = pd.Categorical(df['dia'], categories=week_days)

    return df

def read_files(flist):
    
    for fname,sheet_name in flist:
        df = read_file(fname, sheet_name)
        
    

In [165]:
def plot_litros_por_chofer(df, title=''):
        
    pv = pd.pivot_table(df, values='TOTCARGA', index='NOMCHOFER', aggfunc=['sum'])
    pv = pv.sort_values(('sum', 'TOTCARGA'), ascending=True)
    
    index = pv.index.values
    litros = pv[('sum', 'TOTCARGA')]
    maxvol = litros.max()
    totvol = litros.sum()
    percent = litros/totvol*100
    
    p = figure(y_range=index, 
               plot_width=900, 
               plot_height=1200,
               title=title,
               x_range = (-1,litros.max()*1.2),
               x_axis_label='Carga Total (litros)',
               y_axis_label='Chofer')

    p.hbar(y=index, 
           right=litros, 
           height=0.7, 
           left=0,
           color="firebrick")

    # format x axis to not use scientific notation
    p.axis[0].formatter.use_scientific = False
    
    # insertar porcentaje del total al final de cada barra
    p.text(litros+maxvol/100, index, ['%.1f (%.2f%%)' % (h,r) for (h,r) in zip(litros,percent)],
           text_font_size="6pt", text_align="left", text_baseline="middle")

    show(p)
    
    return p

In [166]:
def plot_vol_por_dia(df, title=''):
    
    dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
    
    pv = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
    
    litros = pv[('sum', 'TOTCARGA')]
    maxvol = litros.max()
    totvol = litros.sum()
    percent = litros/totvol*100

    p = figure(x_range=dias, 
               plot_width=800, 
               plot_height=600,
               title=title,
               x_axis_label='Día de la Semana',
               y_axis_label='Volumen Distribuído (L)')

    p.vbar(x=dias, 
           width=0.5, 
           top=litros, 
           bottom=0,
           color="firebrick")
    
    # format x axis to not use scientific notation
    p.left[0].formatter.use_scientific = False
    
    p.text(dias, litros+maxvol/100, ['%.2f%%' % r for r in percent],
           text_font_size="8pt", text_align="center", text_baseline="bottom")

    show(p)

In [167]:
def plot_vol_trend(dfs, title=''):
    
    dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
    years = ['2017', '2018', '2019', '2020']

    # create data for plotting
    vol_total = {'dias': dias,}
    vol_percent = {'dias': dias,}
    for i, year in enumerate(years):
        pv = pd.pivot_table(dfs[year], values='TOTCARGA', index='dia', aggfunc=['sum'])
        litros = pv[('sum', 'TOTCARGA')]
        maxvol = litros.max()
        totvol = litros.sum()
        percent = litros/totvol*100
        vol_total[year] = litros
        vol_percent[year] = percent
        
    x = [(dia, year) for dia in dias for year in years]
    counts = sum(zip(vol_total))
        
    # create figure and plot data
    p = figure(x_range=dias, 
               plot_width=800, 
               plot_height=600,
               title=title,
               x_axis_label='Día de la Semana',
               y_axis_label='Volumen Distribuído (L)')

    p.vbar(x=dodge(dias, i-2, range=p.x_range), 
           width=0.5, 
           top=litros, 
           bottom=0,
           color="firebrick")
    
    # format x axis to not use scientific notation
    p.left[0].formatter.use_scientific = False
    
    p.text(dias, litros+maxvol/100, ['%.2f%%' % r for r in percent],
           text_font_size="8pt", text_align="center", text_baseline="bottom")

    show(p)
    

In [168]:
dfs = {}
dfs['2017'] = read_file('VIAJ2017.xlsx', 'VIAJ2017')
dfs['2018'] = read_file('VIAJ2018.xlsx', 'VIAJ2018')
dfs['2019'] = read_file('VIAJ2019.xlsx', 'VIAJ2019')
dfs['2020'] = read_file('VIAJ2020.xlsx', 'VIAJ2020')

In [184]:
from bokeh.palettes import GnBu6, OrRd6

dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
years = ['2017', '2018', '2019', '2020']

# create data for plotting
vol_total = {'dias': dias,}
vol_percent = {'dias': dias,}
for i, year in enumerate(years):
    df = dfs[year]
    df = df[df['DEPOSITO'] == 3]
    pv = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
    litros = pv[('sum', 'TOTCARGA')]
    maxvol = litros.max()
    totvol = litros.sum()
    percent = litros/totvol*100
    vol_total[year] = litros.values
    vol_percent[year] = percent.values

x = [(dia, year) for dia in dias for year in years]
counts = sum(zip(vol_total['2017'],vol_total['2018'],vol_total['2019'],vol_total['2020']), ())
source = ColumnDataSource(data=dict(x=x, counts=counts))


# create figure and plot data
p = figure(x_range=FactorRange(*x), 
           plot_width=800, 
           plot_height=600,
           title='',
           x_axis_label='Día de la Semana',
           y_axis_label='Volumen Distribuído (L)')

p.vbar(x='x',  
       top='counts',
       width=0.8,
       bottom=0,
       source=source,
       line_color='white',
       fill_color=factor_cmap('x', palette=GnBu6, factors=years, start=1, end=2))

p.y_range.start = 0
p.y_range.end = max(counts) * 1.2
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

# format x axis to not use scientific notation
p.left[0].formatter.use_scientific = False

p.text(x, counts, ['%.2f' % r for r in counts],
       text_font_size="8pt", text_align="left", text_baseline="middle", angle=np.pi/2, y_offset=-10)

show(p)

In [186]:
from bokeh.palettes import GnBu6, OrRd6
from bokeh.models import LabelSet

dias = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
years = ['2017', '2018', '2019', '2020']

# create data for plotting
vol_total = {'dias': dias,}
vol_percent = {'dias': dias,}
for i, year in enumerate(years):
    df = dfs[year]
    df = df[df['DEPOSITO'] == 3]
    pv = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
    litros = pv[('sum', 'TOTCARGA')]
    maxvol = litros.max()
    totvol = litros.sum()
    percent = litros/totvol*100
    vol_total[year] = litros.values
    vol_percent[year] = percent.values

x = [(dia, year) for dia in dias for year in years]
counts = sum(zip(vol_percent['2017'],vol_percent['2018'],vol_percent['2019'],vol_percent['2020']), ())
source = ColumnDataSource(data=dict(x=x, counts=counts))


# create figure and plot data
p = figure(x_range=FactorRange(*x), 
           plot_width=800, 
           plot_height=600,
           title='',
           x_axis_label='',
           y_axis_label='Porcentaje Volumen Distribuído (%)')

p.vbar(x='x',  
       top='counts',
       width=0.8,
       bottom=0,
       source=source,
       line_color='white',
       fill_color=factor_cmap('x', palette=GnBu6, factors=years, start=1, end=2))

p.y_range.start = 0
p.y_range.end = 30
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

# format x axis to not use scientific notation
p.left[0].formatter.use_scientific = False

p.text(x, counts, ['%.1f%%' % r for r in counts],
       text_font_size="8pt", text_align="left", text_baseline="middle", angle=1.3, y_offset=-2)
#labels = LabelSet(x='x', y='counts', text='counts', level='glyph',
#              x_offset=5, y_offset=5, source=source, render_mode='canvas')
#p.add_layout(labels)

show(p)

In [129]:
df = dfs['2020']
#df = df[df['DEPOSITO'] == 3]
pv = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
litros = pv[('sum', 'TOTCARGA')]
maxvol = litros.max()
totvol = litros.sum()

litros.describe()

count    7.000000e+00
mean     3.515507e+07
std      2.042166e+07
min      0.000000e+00
25%      2.681098e+07
50%      4.114136e+07
75%      4.723365e+07
max      5.685488e+07
Name: (sum, TOTCARGA), dtype: float64

In [128]:
df = dfs['2020']
df = df[df['DEPOSITO'] == 3]
pv = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
litros = pv[('sum', 'TOTCARGA')]
maxvol = litros.max()
totvol = litros.sum()

litros.describe()

count    7.000000e+00
mean     3.338900e+07
std      1.918711e+07
min      0.000000e+00
25%      2.571286e+07
50%      3.886920e+07
75%      4.513051e+07
max      5.316704e+07
Name: (sum, TOTCARGA), dtype: float64

In [135]:
df = dfs['2017']
c0 = df.DEPOSITO.count()
c1 = df.DEPOSITO[df['DEPOSITO'] == 3].count()
c0, c1, c0-c1

(10460, 10460, 0)

In [136]:
df = dfs['2018']
c0 = df.DEPOSITO.count()
c1 = df.DEPOSITO[df['DEPOSITO'] == 3].count()
c0, c1, c0-c1

(10710, 10483, 227)

In [137]:
df = dfs['2019']
c0 = df.DEPOSITO.count()
c1 = df.DEPOSITO[df['DEPOSITO'] == 3].count()
c0, c1, c0-c1

(11858, 10908, 950)

In [138]:
df = dfs['2020']
c0 = df.DEPOSITO.count()
c1 = df.DEPOSITO[df['DEPOSITO'] == 3].count()
c0, c1, c0-c1

(8468, 7801, 667)

In [90]:
sum([z for z in zip([1,2,3],[4,5,6])],())

(1, 4, 2, 5, 3, 6)

In [58]:
p = plot_litros_por_chofer(df17, title='Volumen Total Repartido por cada Chofer en 2017')

In [59]:
plot_vol_por_dia(df17, title='Volúmen Distribuído por Día en 2017')

In [60]:
daily_tc = pd.pivot_table(df17, values='TOTCARGA', index='dia', aggfunc=['sum'])
daily_tc

,sum
,TOTCARGA
dia,
Monday,58214923
Tuesday,54618329
Wednesday,53086339
Thursday,55236585
Friday,73271256
Saturday,8635738
Sunday,0


## 2018

In [61]:
year = 2018
df = read_file('VIAJ2018.xlsx', 'VIAJ2018')

In [62]:
p = plot_litros_por_chofer(df, title='Volumen Total Repartido por cada Chofer en %d' % year)

In [69]:
plot_vol_por_dia(df, title='Volúmen Distribuído por Día en %d' % year)

In [66]:
daily_tc = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
daily_tc

,sum
,TOTCARGA
dia,
Monday,63839408
Tuesday,52270435
Wednesday,55900019
Thursday,56706640
Friday,73697838
Saturday,6519364
Sunday,0


## 2019

In [70]:
year = 2019
df = read_file('VIAJ2019.xlsx', 'VIAJ2019')

In [71]:
p = plot_litros_por_chofer(df, title='Volumen Total Repartido por cada Chofer en %d' % year)

In [72]:
plot_vol_por_dia(df, title='Volúmen Distribuído por Día en %d' % year)

In [73]:
daily_tc = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
daily_tc

,sum
,TOTCARGA
dia,
Monday,70742687
Tuesday,57355356
Wednesday,54844509
Thursday,62192899
Friday,81558719
Saturday,12696394
Sunday,0


## 2020

In [74]:
year = 2020
df = read_file('VIAJ2020.xlsx', 'VIAJ2020')

In [75]:
p = plot_litros_por_chofer(df, title='Volumen Total Repartido por cada Chofer en %d' % year)

In [76]:
plot_vol_por_dia(df, title='Volúmen Distribuído por Día en %d' % year)

In [77]:
daily_tc = pd.pivot_table(df, values='TOTCARGA', index='dia', aggfunc=['sum'])
daily_tc

,sum
,TOTCARGA
dia,
Monday,50905730.0
Tuesday,41141364.0
Wednesday,39092414.0
Thursday,43561570.0
Friday,56854881.0
Saturday,14529537.0
Sunday,0.0
